In [1]:
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import numpy as np
import pandas as pd
import xarray as xr

In [2]:
import datetime as dt
from __future__ import print_function
import requests
import xml.etree.ElementTree as ET
import numpy
import netCDF4

In [3]:
# Author: Unknown
# I got the original version from a word document published by ESGF
# https://docs.google.com/document/d/1pxz1Kd3JHfFp8vR2JCVBfApbsHmbUQQstifhGNdc6U0/edit?usp=sharing
# API AT: https://github.com/ESGF/esgf.github.io/wiki/ESGF_Search_REST_API#results-pagination

def esgf_search(server="https://esgf-node.llnl.gov/esg-search/search",
                files_type="OPENDAP", local_node=True, project="CMIP6",
                verbose=False, format="application%2Fsolr%2Bjson",
                use_csrf=False, **search):
    client = requests.session()
    payload = search
    payload["project"] = project
    payload["type"]= "File"
    if local_node:
        payload["distrib"] = "false"
    if use_csrf:
        client.get(server)
        if 'csrftoken' in client.cookies:
            # Django 1.6 and up
            csrftoken = client.cookies['csrftoken']
        else:
            # older versions
            csrftoken = client.cookies['csrf']
        payload["csrfmiddlewaretoken"] = csrftoken

    payload["format"] = format

    offset = 0
    numFound = 10000
    all_files = []
    files_type = files_type.upper()
    while offset < numFound:
        payload["offset"] = offset
        url_keys = []
        for k in payload:
            url_keys += ["{}={}".format(k, payload[k])]

        url = "{}/?{}".format(server, "&".join(url_keys))
        print(url)
        r = client.get(url)
        r.raise_for_status()
        resp = r.json()["response"]
        numFound = int(resp["numFound"])
        resp = resp["docs"]
        offset += len(resp)
        for d in resp:
            if verbose:
                for k in d:
                    print("{}: {}".format(k,d[k]))
            url = d["url"]
            for f in d["url"]:
                sp = f.split("|")
                if sp[-1] == files_type:
                    all_files.append(sp[0].split(".html")[0])
    return sorted(all_files)

In [4]:
#institution_id="CNRM-CERFACS"
mip_era="CMIP6"
activity_id="HighResMIP"
table_id="Omon"
variable_id="tos"
experiment_id='hist-1950'
institution_id="MPI-M"
source_id="MPI-ESM1-2-HR"
grid_lbl="gn"
member_id="r1i1p1f1"
result = esgf_search(activity_id=activity_id,
                     table_id=table_id,
                     variable_id=variable_id,
                     experiment_id=experiment_id,
                     institution_id=institution_id, 
                     source_id=source_id,
                     grid_label=grid_lbl,
                     member_id=member_id)
# there are mulitple sources of the same data--need to pick one

https://esgf-node.llnl.gov/esg-search/search/?activity_id=HighResMIP&table_id=Omon&variable_id=tos&experiment_id=hist-1950&institution_id=MPI-M&source_id=MPI-ESM1-2-HR&grid_label=gn&member_id=r1i1p1f1&project=CMIP6&type=File&distrib=false&format=application%2Fsolr%2Bjson&offset=0


ConnectionError: HTTPSConnectionPool(host='esgf-node.llnl.gov', port=443): Max retries exceeded with url: /esg-search/search/?activity_id=HighResMIP&table_id=Omon&variable_id=tos&experiment_id=hist-1950&institution_id=MPI-M&source_id=MPI-ESM1-2-HR&grid_label=gn&member_id=r1i1p1f1&project=CMIP6&type=File&distrib=false&format=application%2Fsolr%2Bjson&offset=0 (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x106b79c70>: Failed to establish a new connection: [Errno 8] nodename nor servname provided, or not known'))

In [ ]:
files_to_open = result
print(files_to_open[31:]) #Si empieza en el año 1950 hasta el 1981 es 31 (indice 0)

In [ ]:
# Leemos datos
#ds = xr.open_mfdataset(files_to_open,combine='nested',concat_dim='time',paralell='True')
ds = xr.open_dataset(files_to_open[0])
print(ds.time[-1].values)
#Periodo de tiempo común entre modelo y datos de NOAA
ds=ds.sel(time=slice('1981-12','2014-12'))
print(ds.tos.sizes)

In [ ]:
ds

## Ahora falta seleccionar las diferentes áreas de Estudio: 


# Canarias

In [ ]:
#Canarias:
Ca_bnd_ln=np.array([-25, -5])
Ca_bnd_lt=np.array([10, 45])

#Como lon/lat son matrices 2D necesito una mascara para cortar regiones
#idea: usando el operador 'or' puedo tener las cuatro regiones en una matriz.
msc_ln = (ds.lon>= Ca_bnd_ln[0]) & (ds.lon <= Ca_bnd_ln[1])
msc_lt = (ds.lat>= Ca_bnd_lt[0]) & (ds.lat <= Ca_bnd_lt[1])

Ca_ds = ds.where(msc_ln & msc_lt, drop=True)
Ca_ds


In [ ]:
Ca_ds.lat[:,0].values

In [ ]:
fig=plt.figure(figsize=(14,14))
ax = plt.axes(projection=ccrs.PlateCarree())
ax.coastlines()
cf=ax.contourf(Ca_ds.lon,Ca_ds.lat,
               Ca_ds.tos.mean('time'), cmap='coolwarm')
#plt.plot(Ca_ds.lon,Ca_ds.lat,'o')
fig.colorbar(cf,label='Change in SST (ºC) since 1982', extend='neither',orientation='horizontal');
#Grid de 25km de resolución regular no hace falta interpolar.

In [ ]:
fig=plt.figure(figsize=(14,14))
ax = plt.axes(projection=ccrs.PlateCarree())
ax.coastlines()
cf=ax.contourf(Ca_ds.lon,Ca_ds.lat,
               Ca_ds.tos.sel(time=slice('2014-01','2014-12')).mean('time')
               -Ca_ds.tos.sel(time=slice('1982-01','1982-12')).mean('time'), 
               cmap='coolwarm')
#plt.plot(Ca_ds.lon,Ca_ds.lat,'o')
fig.colorbar(cf,label='Change in SST (ºC) since 1982', extend='neither',orientation='horizontal');
#Grid de 25km de resolución regular no hace falta interpolar.

In [ ]:
print(type(ds.time[0]))

In [ ]:
#Para guardar el .nc, el nombre del fichero sigue la plantilla de los metadatos
#dataset_id_template_ = variable_id_table_id_source_id_experiment_id_member_id_grid_label_YYYYMM-YYYYMM

Fecha_ini=np.datetime_as_string(ds.time[0],unit="D")
Fecha_ini=Fecha_ini.replace("-","")
Fecha_fin=np.datetime_as_string(ds.time[-1],unit="D")
Fecha_fin=Fecha_fin.replace("-","")

FileIn=f'CaCLME_{variable_id}_{table_id}_{source_id}_{experiment_id}_{member_id}_{grid_lbl}_{Fecha_ini}_{Fecha_fin}.nc' 
FileIn
Ca_ds.to_netcdf(FileIn) # lon lat recortados

Cells para OMIP porque el datatype es .cftime. En el resto de productos es tipo numpy.datetime64

In [ ]:
#Para guardar el .nc, el nombre del fichero sigue la plantilla de los metadatos
#dataset_id_template_ = variable_id_table_id_source_id_experiment_id_member_id_grid_label_YYYYMM-YYYYMM

#print(ds.time[0].dt.strftime("%d%m%Y").values)
#Fecha_ini=ds.time[0].dt.strftime("%Y%m%d").values

#Fecha_fin=ds.time[-1].dt.strftime("%Y%m%d").values

#FileIn=f'CaCLME_{variable_id}_{table_id}_{source_id}_{experiment_id}_{member_id}_{grid_lbl}_{Fecha_ini}_{Fecha_fin}.nc' 
#print(FileIn)
#Ca_ds.to_netcdf(FileIn) # lon lat recortados

# California

In [ ]:
Cl_bnd_ln=np.array([ -131,-110])
Cl_bnd_lt=np.array([10, 45])

#Como lon/lat son matrices 2D necesito una mascara para cortar regiones
#idea: usando el operador 'or' puedo tener las cuatro regiones en una matriz.
msc_ln = (ds.lon>= Cl_bnd_ln[0]) & (ds.lon <= Cl_bnd_ln[1])
msc_lt = (ds.lat>= Cl_bnd_lt[0]) & (ds.lat <= Cl_bnd_lt[1])

Cl_ds = ds.where(msc_ln & msc_lt, drop=True)
Cl_ds 

In [ ]:
fig=plt.figure(figsize=(14,14))

ax = plt.axes(projection=ccrs.PlateCarree())
ax.coastlines()

cf=ax.contourf(Cl_ds.lon,Cl_ds.lat,
               Cl_ds.tos.mean('time'), cmap='coolwarm')

fig.colorbar(cf,label='Change in SST (ºC) since 1982', extend='neither',orientation='horizontal');
#Grid de 25km de resolución regular no hace falta interpolar.

In [ ]:
fig=plt.figure(figsize=(14,14))

ax = plt.axes(projection=ccrs.PlateCarree())
ax.coastlines()

cf=ax.contourf(Cl_ds.lon,Cl_ds.lat,
               Cl_ds.tos.sel(time=slice('2018-01','2018-12')).mean('time')
               -Cl_ds.tos.sel(time=slice('1982-01','1982-12')).mean('time'), cmap='coolwarm')

fig.colorbar(cf,label='Change in SST (ºC) since 1982', extend='neither',orientation='horizontal');
#Grid de 25km de resolución regular no hace falta interpolar.

In [ ]:
#Para guardar el .nc, el nombre del fichero sigue la plantilla de los metadatos
#dataset_id_template_ = variable_id_table_id_source_id_experiment_id_member_id_grid_label_YYYYMM-YYYYMM

FileIn=f'ClCLME_{variable_id}_{table_id}_{source_id}_{experiment_id}_{member_id}_{grid_lbl}_{Fecha_ini}_{Fecha_fin}.nc' 
FileIn
Cl_ds.to_netcdf(FileIn) # lon lat recortados

# Benguela

In [ ]:
Be_bnd_ln=np.array([0,20])
Be_bnd_lt=np.array([-45,-10])

#Como lon/lat son matrices 2D necesito una mascara para cortar regiones
#idea: usando el operador 'or' puedo tener las cuatro regiones en una matriz.

msc_ln = (ds.lon>= Be_bnd_ln[0]) & (ds.lon <= Be_bnd_ln[1])
msc_lt = (ds.lat>= Be_bnd_lt[0]) & (ds.lat <= Be_bnd_lt[1])

Be_ds = ds.where(msc_ln & msc_lt, drop=True)
Be_ds 

In [ ]:
fig=plt.figure(figsize=(14,14))

ax = plt.axes(projection=ccrs.PlateCarree())
ax.coastlines()

cf=ax.contourf(Be_ds.lon,Be_ds.lat,
               Be_ds.tos.mean('time'), cmap='coolwarm')

fig.colorbar(cf,label='Change in SST (ºC) since 1982', extend='neither',orientation='horizontal');
#Grid de 25km de resolución regular no hace falta interpolar.

In [ ]:
fig=plt.figure(figsize=(14,14))

ax = plt.axes(projection=ccrs.PlateCarree())
ax.coastlines()

cf=ax.contourf(Be_ds.lon,Be_ds.lat,
               Be_ds.tos.sel(time=slice('2014-01','2014-12')).mean('time')
               -Be_ds.tos.sel(time=slice('1982-01','1982-12')).mean('time'), cmap='coolwarm')

fig.colorbar(cf,label='Change in SST (ºC) since 1982', extend='neither',orientation='horizontal');
#Grid de 25km de resolución regular no hace falta interpolar.

In [ ]:
#Para guardar el .nc, el nombre del fichero sigue la plantilla de los metadatos
#dataset_id_template_ = variable_id_table_id_source_id_experiment_id_member_id_grid_label_YYYYMM-YYYYMM

FileIn=f'BeCLME_{variable_id}_{table_id}_{source_id}_{experiment_id}_{member_id}_{grid_lbl}_{Fecha_ini}_{Fecha_fin}.nc' 
FileIn
Be_ds.to_netcdf(FileIn) # lon lat recortados

# Humboldt

In [ ]:
Hu_bnd_ln=np.array([-90,-68])
Hu_bnd_lt=np.array([-45,-10])

#Como lon/lat son matrices 2D necesito una mascara para cortar regiones
#idea: usando el operador 'or' puedo tener las cuatro regiones en una matriz.

msc_ln = (ds.lon>= Hu_bnd_ln[0]) & (ds.lon <= Hu_bnd_ln[1])
msc_lt = (ds.lat>= Hu_bnd_lt[0]) & (ds.lat <= Hu_bnd_lt[1])

Hu_ds = ds.where(msc_ln & msc_lt, drop=True)
Hu_ds 

In [ ]:
fig=plt.figure(figsize=(14,14))

ax = plt.axes(projection=ccrs.PlateCarree())
ax.coastlines()

cf=ax.contourf(Hu_ds.lon,Hu_ds.lat,
               Hu_ds.tos.mean('time'), cmap='coolwarm')

fig.colorbar(cf,label='Change in SST (ºC) since 1982', extend='neither',orientation='horizontal');
#Grid de 25km de resolución regular no hace falta interpolar.

In [ ]:
fig=plt.figure(figsize=(14,14))

ax = plt.axes(projection=ccrs.PlateCarree())
ax.coastlines()

cf=ax.contourf(Hu_ds.lon,Hu_ds.lat,
               Hu_ds.tos.sel(time=slice('2018-01','2018-12')).mean('time')
               -Hu_ds.tos.sel(time=slice('1982-01','1982-12')).mean('time'), cmap='coolwarm')

fig.colorbar(cf,label='Change in SST (ºC) since 1982', extend='neither',orientation='horizontal');
#Grid de 25km de resolución regular no hace falta interpolar.

In [ ]:
#Para guardar el .nc, el nombre del fichero sigue la plantilla de los metadatos
#dataset_id_template_ = variable_id_table_id_source_id_experiment_id_member_id_grid_label_YYYYMM-YYYYMM

FileIn=f'HuCLME_{variable_id}_{table_id}_{source_id}_{experiment_id}_{member_id}_{grid_lbl}_{Fecha_ini}_{Fecha_fin}.nc' 
FileIn
Hu_ds.to_netcdf(FileIn) # lon lat recortados